In [ ]:
!pip install git+https://github.com/LIAAD/yake  #Установка библиотеки YAKE из гитхаба

  Cloning https://github.com/LIAAD/yake to /tmp/pip-req-build-c5x8f5m0
  Running command git clone --filter=blob:none --quiet https://github.com/LIAAD/yake /tmp/pip-req-build-c5x8f5m0
  Resolved https://github.com/LIAAD/yake to commit 0fa58cceb465162b6bd0cab7ec967edeb907fbcc
  Preparing metadata (setup.py) ... done
  Created wheel for yake: filename=yake-0.4.8-py2.py3-none-any.whl size=61997 sha256=8b916d70056267d51163fea5cd7636bed3bd4a9ea740d8a362e09c0d90742e0a
  Stored in directory: /tmp/pip-ephem-wheel-cache-gmwj6ecr/wheels/06/e6/1c/8f614adcd4b053020d672b9cbc5ef1166407755a4b71adea0d
Successfully built yake


In [ ]:
import os
import zipfile
import yake

1. Загрузить и распаковать архив

In [ ]:
from google.colab import files
uploaded = files.upload()
#Чтобы исключить ошибки, лучше загрузить ZIP через код

Saving IMS2013-2024-20250413T075016Z-001 (1).zip to IMS2013-2024-20250413T075016Z-001 (1).zip


In [ ]:
zip_path = "IMS2013-2024-20250413T075016Z-001 (1).zip" #прописываем путь до него
extract_dir = "IMS_extracted" #создаем папку, куда мы хотим распаковать наш архив

In [ ]:
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir) #извлекаем все файлы из архива в папку IMS_extracted, которая появится слева в разделе Files.

2. Прошлые опыты показали, что YAKE! лучше всего справляется с КВ. Устанавливаем и приминеям его


In [ ]:
language = "ru"
max_ngram_size = 3
deduplication_threshold = 0.9
deduplication_algo = 'seqm'
windowSize = 1
numOfKeywords = 10

kw_extractor = yake.KeywordExtractor(
    lan=language,
    n=max_ngram_size,
    dedupLim=deduplication_threshold,
    dedupFunc=deduplication_algo,
    windowsSize=windowSize,
    top=numOfKeywords,
    features=None
)

В архиве все аннотации хранятся в txt файлах, которые заканчиваются на Abstract_rus.txt
Обработаем их


In [ ]:
results = {}

for root, dirs, files in os.walk(extract_dir):
    for file_name in files:
        if file_name.endswith("Abstract_rus.txt"):
            file_path = os.path.join(root, file_name)
            with open(file_path, 'r', encoding='utf-8') as file:
                text = file.read()
                keywords = kw_extractor.extract_keywords(text)
                results[file_name] = [kw[0] for kw in keywords]

# Вывести результат
for filename, kws in results.items():
    print(f"\n{filename}:")
    for kw in kws:
        print(f" - {kw}")


Vybornaya_IMS_2024_Abstract_rus.txt:
 - исследуется синтаксическая неоднозначность
 - синтаксическая неоднозначность предложных
 - неоднозначность предложных конструкций
 - характеристики соотношения предлогов
 - статье описываются частотные
 - описываются частотные характеристики
 - частотные характеристики соотношения
 - синтаксически неоднозначных предложений
 - данных предложных конструкций
 - предложных конструкций

Belkin_IMS_2024_Abstract_rus.txt:
 - Пользовательский фидбек является
 - фидбек является ценным
 - является ценным источником
 - ценным источником информации
 - выявления проблемных аспектов
 - оценки качества услуг
 - Пользовательский фидбек
 - частности поликлиниками
 - проблемных аспектов
 - фидбек является

MitrofanovaAdamova_IMS_2024_Abstract_rus.txt:
 - Статья посвящена проблемам
 - математической лингвистики СПбГУ
 - посвящена проблемам разработки
 - проблемам разработки корпуса
 - кафедре математической лингвистики
 - разработки корпуса статей
 - Статья посвящ